In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import pathlib
from copy import copy
from matplotlib import cm, colors
import cv2

bins = [5,10,20,40]

baseline05 = 0.966
baseline025 = 0.95
baseline01 = 0.819

baseline = baseline05
dtresult = pd.read_csv('./Results/A/DT-0.5-os-3-usr-1-pm-1.csv',
    names=['type','sharpening','windowSize','neighbourhood','buckets','histnormalise','k-nearest','cv-auc','test-auc']
    , header=1)
knnresult = pd.read_csv('./Results/A/KNN-0.5-os-3-usr-1-pm-1.csv',
    names=['type','sharpening','windowSize','neighbourhood','buckets','histnormalise','k-nearest','cv-auc','test-auc']
    , header=1)
dtresult.insert(1, 'Classifier', 'DT')
knnresult.insert(1, 'Classifier', 'KNN')
masterdf = pd.concat([dtresult, knnresult])
masterdf.insert(1, "Threshold", 0.5)
masterdf = masterdf.sort_values(by=['cv-auc'], ascending=False)

####################
# Done with 0.5
####################

baseline = baseline025
dtresult = pd.read_csv('./Results/A/DT-0.25-os-3-usr-1-pm-1.csv',
    names=['type','sharpening','windowSize','neighbourhood','buckets','histnormalise','k-nearest','cv-auc','test-auc']
    , header=1)
knnresult = pd.read_csv('./Results/A/KNN-0.25-os-3-usr-1-pm-1.csv',
    names=['type','sharpening','windowSize','neighbourhood','buckets','histnormalise','k-nearest','cv-auc','test-auc']
    , header=1)
dtresult.insert(1, 'Classifier', 'DT')
knnresult.insert(1, 'Classifier', 'KNN')
dtresult.insert(1, 'Threshold', 0.25)
knnresult.insert(1, 'Threshold', 0.25)
masterdf = pd.concat([masterdf, knnresult, dtresult])
masterdf = masterdf.sort_values(by=['cv-auc'], ascending=False)

####################
# Done with 0.25
####################

baseline = baseline01
dtresult = pd.read_csv('./Results/A/DT-0.1-os-3-usr-1-pm-1.csv',
    names=['type','sharpening','windowSize','neighbourhood','buckets','histnormalise','k-nearest','cv-auc','test-auc']
    , header=1)
knnresult = pd.read_csv('./Results/A/KNN-0.1-os-3-usr-1-pm-1.csv',
    names=['type','sharpening','windowSize','neighbourhood','buckets','histnormalise','k-nearest','cv-auc','test-auc']
    , header=1)
dtresult.insert(1, 'Classifier', 'DT')
knnresult.insert(1, 'Classifier', 'KNN')
dtresult.insert(1, 'Threshold', 0.1)
knnresult.insert(1, 'Threshold', 0.1)
masterdf = pd.concat([masterdf, knnresult, dtresult])
masterdf = masterdf.sort_values(by=['cv-auc'], ascending=False)

####################
# Done with 0.10
####################

baselines = {0.1: 0.819, 0.25: 0.95, 0.5: 0.966}

print("   --- CV based on Z ---")
for threshold in [0.1, 0.25, 0.5]:
    for classifier in ['DT', 'KNN']:
        for Z in [1,3,5,7]:
            res = masterdf[(masterdf['Classifier'] == classifier) & (masterdf['Threshold'] == threshold) & (masterdf['windowSize'] == Z)]['test-auc'].iloc[0]
            print(f'{threshold} {classifier} {Z} {res} {res - baselines[threshold]}')

print("")
print("   --- CV based on XY ---")

for threshold in [0.1, 0.25, 0.5]:
    for classifier in ['DT', 'KNN']:
        for XY in [3,5,7]:
            res = masterdf[(masterdf['Classifier'] == classifier) & (masterdf['Threshold'] == threshold) & (masterdf['neighbourhood'] == XY)]['test-auc'].iloc[0]
            print(f'{threshold} {classifier} {XY} {res} {res - baselines[threshold]}')

print("")
print("   --- CV based on bins ---")

for threshold in [0.1, 0.25, 0.5]:
    for classifier in ['DT', 'KNN']:
        for bins in [5,10,20,40]:
            res = masterdf[(masterdf['Classifier'] == classifier) & (masterdf['Threshold'] == threshold) & (masterdf['buckets'] == bins)]['test-auc'].iloc[0]
            print(f'{threshold} {classifier} {bins} {res} {res - baselines[threshold]}')

print("")
print("   --- CV based on XYZ ---")

for threshold in [0.1, 0.25, 0.5]:
    for classifier in ['DT', 'KNN']:
        for Z in [1,3,5,7]:
            for XY in [3,5,7]:
                res = masterdf[(masterdf['Classifier'] == classifier) & (masterdf['Threshold'] == threshold) & (masterdf['windowSize'] == Z) & (masterdf['neighbourhood'] == XY)]['test-auc'].iloc[0]
                print(f'{threshold} {classifier} {Z} {XY} {res} {res - baselines[threshold]}')

columns = ['Threshold','Classifier', 'Type', 'Z', 'XY', 'Bins', 'Hyperparameter', 'ROC-AUC']
gooddf = pd.DataFrame(columns=columns)
for threshold in [0.1, 0.25, 0.5]:
    for classifier in ['DT', 'KNN']:
        for type in ['moran', 'scatter', 'spatstat']:
            for Z in [1,3,5,7]:
                for XY in [3,5,7]:
                    row = masterdf[(masterdf['Classifier'] == classifier)
                        & (masterdf['Threshold'] == threshold)
                        & (masterdf['type'] == type)
                        & (masterdf['windowSize'] == Z)
                        & (masterdf['neighbourhood'] == XY)].iloc[0]
                    nextRow =  pd.DataFrame([[threshold, classifier, type, Z, XY, row['buckets'], row['k-nearest'], row['test-auc']]], columns=columns)
                    gooddf = pd.concat([gooddf, nextRow])

gooddf = gooddf.sort_values(by=['ROC-AUC'], ascending=False)

finaldf = pd.DataFrame(columns=columns)
print("This is for the plots")
for threshold in [0.1, 0.25, 0.5]:
    print(gooddf[gooddf['Threshold'] == threshold].head(3))
    for classifier in ['DT', 'KNN']:
        for type in ['moran', 'scatter', 'spatstat']:
            row = gooddf[(gooddf['Classifier'] == classifier)
                & (gooddf['Threshold'] == threshold)
                & (gooddf['Type'] == type)].iloc[0]
            nextRow =  pd.DataFrame([[threshold, classifier, type, row['Z'], row['XY'], row['Bins'], row['Hyperparameter'], row['ROC-AUC']]], columns=columns)
            finaldf = pd.concat([finaldf, nextRow])

print("This is for the table")
print(finaldf)

   --- CV based on Z ---
0.1 DT 1 0.8303439803439804 0.011343980343980431
0.1 DT 3 0.784889434889435 -0.03411056511056498
0.1 DT 5 0.7945945945945946 -0.024405405405405367
0.1 DT 7 0.8271498771498771 0.008149877149877138
0.1 KNN 1 0.9232186732186732 0.10421867321867329
0.1 KNN 3 0.8249385749385749 0.00593857493857497
0.1 KNN 5 0.8090909090909091 -0.009909090909090867
0.1 KNN 7 0.8572481572481573 0.03824815724815733
0.25 DT 1 0.93188202247191 -0.018117977528089968
0.25 DT 3 0.8456460674157303 -0.10435393258426962
0.25 DT 5 0.8723314606741575 -0.0776685393258425
0.25 DT 7 0.7981741573033708 -0.15182584269662913
0.25 KNN 1 0.9521067415730338 0.00210674157303381
0.25 KNN 3 0.9668539325842698 0.016853932584269815
0.25 KNN 5 0.8973314606741574 -0.05266853932584259
0.25 KNN 7 0.9525280898876404 0.0025280898876404168
0.5 DT 1 0.9144265232974912 -0.05157347670250878
0.5 DT 3 0.9229390681003584 -0.04306093189964155
0.5 DT 5 0.8288530465949822 -0.1371469534050178
0.5 DT 7 0.8620071684587813 -0.10

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import pathlib
from copy import copy
from matplotlib import cm, colors
import cv2

bins = [5,10,20,40]

baseline05 = 0.966
baseline025 = 0.95
baseline01 = 0.819

baseline = baseline05
dtresult = pd.read_csv('./Results/B/DT-0.5-os-1-usr-1-pm-1.csv',
    names=['type','sharpening','windowSize','neighbourhood','buckets','histnormalise','k-nearest','cv-auc','test-auc']
    , header=1)
knnresult = pd.read_csv('./Results/B/KNN-0.5-os-1-usr-1-pm-1.csv',
    names=['type','sharpening','windowSize','neighbourhood','buckets','histnormalise','k-nearest','cv-auc','test-auc']
    , header=1)
dtresult.insert(1, 'Classifier', 'DT')
knnresult.insert(1, 'Classifier', 'KNN')
masterdf = pd.concat([dtresult, knnresult])
masterdf.insert(1, "Threshold", 0.5)
masterdf = masterdf.sort_values(by=['cv-auc'], ascending=False)

####################
# Done with 0.5
####################

baseline = baseline025
dtresult = pd.read_csv('./Results/B/DT-0.25-os-1-usr-1-pm-1.csv',
    names=['type','sharpening','windowSize','neighbourhood','buckets','histnormalise','k-nearest','cv-auc','test-auc']
    , header=1)
knnresult = pd.read_csv('./Results/B/KNN-0.25-os-1-usr-1-pm-1.csv',
    names=['type','sharpening','windowSize','neighbourhood','buckets','histnormalise','k-nearest','cv-auc','test-auc']
    , header=1)
dtresult.insert(1, 'Classifier', 'DT')
knnresult.insert(1, 'Classifier', 'KNN')
dtresult.insert(1, 'Threshold', 0.25)
knnresult.insert(1, 'Threshold', 0.25)
masterdf = pd.concat([masterdf, knnresult, dtresult])
masterdf = masterdf.sort_values(by=['cv-auc'], ascending=False)

####################
# Done with 0.25
####################

baseline = baseline01
dtresult = pd.read_csv('./Results/B/DT-0.1-os-1-usr-1-pm-1.csv',
    names=['type','sharpening','windowSize','neighbourhood','buckets','histnormalise','k-nearest','cv-auc','test-auc']
    , header=1)
knnresult = pd.read_csv('./Results/B/KNN-0.1-os-1-usr-1-pm-1.csv',
    names=['type','sharpening','windowSize','neighbourhood','buckets','histnormalise','k-nearest','cv-auc','test-auc']
    , header=1)
dtresult.insert(1, 'Classifier', 'DT')
knnresult.insert(1, 'Classifier', 'KNN')
dtresult.insert(1, 'Threshold', 0.1)
knnresult.insert(1, 'Threshold', 0.1)
masterdf = pd.concat([masterdf, knnresult, dtresult])
masterdf = masterdf.sort_values(by=['cv-auc'], ascending=False)
# masterdf["test-auc"] = pd.to_numeric(masterdf["test-auc"])

####################
# Done with 0.10
####################

baselines = {0.1: 0.819, 0.25: 0.95, 0.5: 0.966}

print("   --- CV based on Z ---")
for threshold in [0.1, 0.25, 0.5]:
    for classifier in ['DT', 'KNN']:
        for Z in [1,3,5,7]:
            res = masterdf[(masterdf['Classifier'] == classifier) & (masterdf['Threshold'] == threshold) & (masterdf['windowSize'] == Z)]['test-auc'].iloc[0]
            print(f'{threshold} {classifier} {Z} {res} {res - baselines[threshold]}')

print("")
print("   --- CV based on XY ---")

for threshold in [0.1, 0.25, 0.5]:
    for classifier in ['DT', 'KNN']:
        for XY in [3,5,7]:
            res = masterdf[(masterdf['Classifier'] == classifier) & (masterdf['Threshold'] == threshold) & (masterdf['neighbourhood'] == XY)]['test-auc'].iloc[0]
            print(f'{threshold} {classifier} {XY} {res} {res - baselines[threshold]}')

print("")
print("   --- CV based on bins ---")

for threshold in [0.1, 0.25, 0.5]:
    for classifier in ['DT', 'KNN']:
        for bins in [5,10,20,40]:
            res = masterdf[(masterdf['Classifier'] == classifier) & (masterdf['Threshold'] == threshold) & (masterdf['buckets'] == bins)]['test-auc'].iloc[0]
            print(f'{threshold} {classifier} {bins} {res} {res - baselines[threshold]}')

print("")
print("   --- CV based on XYZ ---")

for threshold in [0.1, 0.25, 0.5]:
    for classifier in ['DT', 'KNN']:
        for Z in [1,3,5,7]:
            for XY in [3,5,7]:
                res = masterdf[(masterdf['Classifier'] == classifier) & (masterdf['Threshold'] == threshold) & (masterdf['windowSize'] == Z) & (masterdf['neighbourhood'] == XY)]['test-auc'].iloc[0]
                print(f'{threshold} {classifier} {Z} {XY} {res} {res - baselines[threshold]}')

columns = ['Threshold','Classifier', 'Type', 'Z', 'XY', 'Bins', 'Hyperparameter', 'ROC-AUC']
gooddf = pd.DataFrame(columns=columns)
for threshold in [0.1, 0.25, 0.5]:
    for classifier in ['DT', 'KNN']:
        for type in ['moran', 'scatter', 'spatstat']:
            for Z in [1,3,5,7]:
                for XY in [3,5,7]:
                    row = masterdf[(masterdf['Classifier'] == classifier)
                        & (masterdf['Threshold'] == threshold)
                        & (masterdf['type'] == type)
                        & (masterdf['windowSize'] == Z)
                        & (masterdf['neighbourhood'] == XY)].iloc[0]
                    nextRow =  pd.DataFrame([[threshold, classifier, type, Z, XY, row['buckets'], row['k-nearest'], row['test-auc']]], columns=columns)
                    gooddf = pd.concat([gooddf, nextRow])

gooddf = gooddf.sort_values(by=['ROC-AUC'], ascending=False)

finaldf = pd.DataFrame(columns=columns)
print("This is for the plots of B")
for threshold in [0.1, 0.25, 0.5]:
    print(gooddf[gooddf['Threshold'] == threshold].head(3))
    for classifier in ['DT', 'KNN']:
        for type in ['moran', 'scatter', 'spatstat']:
            row = gooddf[(gooddf['Classifier'] == classifier)
                & (gooddf['Threshold'] == threshold)
                & (gooddf['Type'] == type)].iloc[0]
            nextRow =  pd.DataFrame([[threshold, classifier, type, row['Z'], row['XY'], row['Bins'], row['Hyperparameter'], row['ROC-AUC']]], columns=columns)
            finaldf = pd.concat([finaldf, nextRow])

print("This is for the table B")
print(finaldf)


   --- CV based on Z ---
0.1 DT 1 0.7625 -0.056499999999999995
0.1 DT 3 0.6166666666666667 -0.20233333333333325
0.1 DT 5 0.5802083333333333 -0.23879166666666662
0.1 DT 7 0.7020833333333334 -0.11691666666666656
0.1 KNN 1 0.9017361111111112 0.08273611111111123
0.1 KNN 3 0.7753472222222222 -0.04365277777777776
0.1 KNN 5 0.7600694444444445 -0.058930555555555486
0.1 KNN 7 0.7041666666666666 -0.11483333333333334
0.25 DT 1 0.8706896551724138 -0.07931034482758614
0.25 DT 3 0.6905172413793103 -0.2594827586206897
0.25 DT 5 0.7655172413793104 -0.18448275862068952
0.25 DT 7 0.6913793103448276 -0.2586206896551724
0.25 KNN 1 0.9754310344827586 0.02543103448275863
0.25 KNN 3 0.8038793103448275 -0.14612068965517244
0.25 KNN 5 0.793103448275862 -0.15689655172413797
0.25 KNN 7 0.8116379310344828 -0.13836206896551717
0.5 DT 1 0.6444444444444445 -0.3215555555555555
0.5 DT 3 0.7305555555555555 -0.23544444444444446
0.5 DT 5 0.7416666666666666 -0.22433333333333338
0.5 DT 7 0.8666666666666667 -0.0993333333333